Metric testing
===

In [1]:
import datasets
import evaluate

from experiment import metrics

In [2]:
# https://github.com/huggingface/evaluate/issues/428
bleurt = evaluate.load(
    "bleurt", "bleurt-base-512", module_type="metric", download_config=datasets.DownloadConfig(use_etag=False)
)

INFO:tensorflow:Reading checkpoint /Users/zacharylevonian/.cache/huggingface/datasets/downloads/extracted/0b5f615fbe4df81a585448a4e6f47b4bb3af737cc290a4d96effa6ef1840ea73/bleurt-base-512.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [3]:
predictions = ["hello there", "general skywalker"]
references = ["hello there", "general kenobi"]
bleurt.compute(predictions=predictions, references=references)

{'scores': [1.175394892692566, -1.1031553745269775]}

In [4]:
def compute_bleurt(passages: list[str], generation: str):
    references = passages + [
        "\n".join(passages),
    ]
    predictions = [
        generation,
    ] * (len(passages) + 1)
    scores = bleurt.compute(predictions=predictions, references=references)["scores"]
    return max(scores)


compute_bleurt(["The alphabet is 26 letters long.", "Math is not so easy."], "The English alphabet is 26 letters long.")

0.4088791608810425

In [5]:
bert_score = metrics.get_bertscore_metric_object()

In [7]:
predictions = ["hello there", "general skywalker"]
references = ["hello there", "general kenobi"]
bert_score.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.9999998807907104, 0.9180971384048462],
 'recall': [0.9999998807907104, 0.8901697397232056],
 'f1': [0.9999998807907104, 0.9039177894592285],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.33.1)'}

In [9]:
# must match counts
bert_score.compute(
    predictions=["This is a test."] * 2,
    references=["Two reference sentences.", "Second is a test sentence."],
    lang="en",
)

{'precision': [0.8900542259216309, 0.9747406840324402],
 'recall': [0.8820334672927856, 0.9553087949752808],
 'f1': [0.8860256671905518, 0.9649269580841064],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.33.1)'}